# Real time predistortion demonstration

## Load PycQED

In [2]:
from pycqed.instrument_drivers.physical_instruments.ZurichInstruments import ZI_HDAWG8

## Connect to the HDAWG device

Please replace 'dev8018' by the correct device identifier.

In [ ]:
AWG8 = ZI_HDAWG8.ZI_HDAWG8('AWG8_8018', device='dev8018')

## Upload SeqC program

In [3]:
AWG8.configure_awg_from_string(0, '''
// Constants
const FS = 2.4e9;
const PULSE_AMP = 0.5;
const PULSE_LENGTH = 5e-6;

// Waveform
wave w = join(PULSE_AMP*ones(PULSE_LENGTH*FS), zeros(PULSE_LENGTH*FS), -PULSE_AMP*ones(PULSE_LENGTH*FS));

while (1) {
    // Play waveform on channel 1
    playWave(1, w);
    setTrigger(1);
    setTrigger(0);
}    
''')

Configuring AWG_nr 0.
Compilation started
Detected 1 HDAWG sub-devices
Compiling string
Compilation successful
Uploading file to device...
File successfully uploaded in 1.83s


## Enable output channel 1

In [4]:
AWG8.set('sigouts_0_on', 1)

## Start the AWG

In [5]:
AWG8.start()

## Run the real-time pre-distortion on channel 1

### High-pass filter compensation (HPC)

* Exponential decay is compensated by linear increase.
* Mapping from input $x_n$ to output $y_n$ for each sample with index $n$:
  * $y_n = x_n + k(2u_n-x_n)$.
* The state variable is $u_n = u_{n-8} + \bar{x}_n$, where $\bar{x}_n$ is an average over eight parallel paths. 
* Parameter $k = 1/(2\tau f_s$), where $\tau$ is the time constant and $f_s = 2.4\;\mathrm{GSa/s}$ is the sampling rate.
* Important: the state variable is only updated every 8th clock cycle of the sample clock, i.e with rate $f_s/8 = 300\;\mathrm{MSa/s}$


In [13]:
AWG8.run_realtime_predistortion(0, hpc_tau = 20e-6)

{'delay_clk_cycles': 17, 'hpc_tau': 2.000144922931213e-05, 'exp_coefs': []}

## Disable real-time pre-distortion again

In [16]:
AWG8.run_realtime_predistortion(0)

{'delay_clk_cycles': 0, 'hpc_tau': None, 'exp_coefs': []}

### Exponential under- / overshoot compensation (EXP)
* Mapping from input $x_n$ to output $y_n$ for each sample with index $n$:
   * $y_n = (1 - k) x_n - k u_n$
* The state variable is an exponential moving average: $u_n = u_{n-32} + 32\alpha (\bar{x}_n - u_{n-32})$, where $\bar{x}_n$ is an average over 32 consecutive samples. 
* Important: the state variable is only updated every 8th clock cycle of the sample clock, i.e with rate $f_s/8 = 300\;\mathrm{MSa/s}$
* Parameters: 
 * $\alpha = 1 - e^{-1/(f_s\tau(1+A)}$
 * $k = \begin{cases}A/(1 + A - \alpha) & \text{, for } A\geq0\\ A/((1+A)(1-\alpha)) & \text{, for } A<0\end{cases}$
 * where $\tau$ is the time constant and $A$ the amplitude of the over- / undershoot relative to the pulse amplitude.

In [15]:
AWG8.run_realtime_predistortion(0, hpc_tau = 20e-6, exp_coefs=[
    {'tau' : 2e-6, 'amp' : -0.01}
])

{'delay_clk_cycles': 31,
 'hpc_tau': 2.000144922931213e-05,
 'exp_coefs': [{'tau': 1.9999999949504854e-06, 'amp': -0.009999999776482582}]}

### Multiple EXP filters

In [11]:
AWG8.run_realtime_predistortion(0, hpc_tau = 20e-6, exp_coefs=[
    {'tau' : 2e-6, 'amp' : 0.01},
    {'tau' : 2e-6, 'amp' : -0.01}
])

{'delay_clk_cycles': 45,
 'hpc_tau': 2.000144922931213e-05,
 'exp_coefs': [{'tau': 1.9999999949504854e-06, 'amp': 0.009999999776482582},
  {'tau': 1.9999999949504854e-06, 'amp': -0.009999999776482582}]}

In [19]:
AWG8.run_realtime_predistortion(0, hpc_tau = 20e-6, exp_coefs=[
    {'tau' : 2e-6, 'amp' : 0.01},
    {'tau' : 1e-6, 'amp' : -0.005}
])

{'delay_clk_cycles': 45,
 'hpc_tau': 2.000144922931213e-05,
 'exp_coefs': [{'tau': 1.9999999949504854e-06, 'amp': 0.009999999776482582},
  {'tau': 9.999999974752427e-07, 'amp': -0.004999999888241291}]}